In [1]:
import numpy as np
import pandas as pd

import xgboost

import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

from sklearn.feature_selection import SelectKBest
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

df = pd.read_csv('train.csv')
df.drop('Id',axis=1, inplace=True)
df_test = pd.read_csv('test.csv')
df_test.drop('Id',axis=1, inplace=True)

In [ ]:
df_numeric = df.select_dtypes(exclude=['object'])
df_cat = df.select_dtypes(include=['object'])

In [5]:
df_cat['Alley'].fillna('No', inplace=True)
df_cat['BsmtQual'].fillna('No', inplace=True)
df_cat['BsmtCond'].fillna('No', inplace=True)
df_cat['BsmtExposure'].fillna('No', inplace=True)
df_cat['MasVnrType'].fillna('No', inplace=True)
df_cat['Electrical'].fillna('SBrkr', inplace=True)
df_cat['FireplaceQu'].fillna('No', inplace=True)
df_cat['GarageType'].fillna('No', inplace=True)
df_cat['GarageFinish'].fillna('No', inplace=True)
df_cat['GarageQual'].fillna('No', inplace=True)
df_cat['GarageCond'].fillna('No', inplace=True)
df_cat['PoolQC'].fillna('No', inplace=True)
df_cat['Fence'].fillna('No', inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_cat['BsmtFinType1'])
le.classes_
BsmtFinType1 = le.transform(df_cat['BsmtFinType1'])
BsmtFinType1_df = pd.DataFrame(BsmtFinType1, columns=['BsmtFinType1_enum'])

le.fit(df_cat['BsmtFinType2'])
le.classes_
BsmtFinType2 = le.transform(df_cat['BsmtFinType2'])
BsmtFinType2_df = pd.DataFrame(BsmtFinType2, columns=['BsmtFinType2_enum'])

MiscFeature = pd.get_dummies(df_cat['MiscFeature'], columns=['MiscFeature'])

df_combined = pd.concat([BsmtFinType1_df, BsmtFinType2_df, MiscFeature], axis=1)

In [ ]:
df = pd.concat([df_combined,df_cat], axis=1)

# Функция для обработки категориальных признаков
def encode_categorical_features(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            unique_values = df[column].nunique()
            if unique_values <= 5:
                # Применяем one-hot encoding
                df = pd.get_dummies(df, columns=[column])
            else:
                # Применяем label encoding
                le = LabelEncoder()
                df[column] = le.fit_transform(df[column])
    return df

# Применяем функцию к нашему DataFrame
df_encoded = encode_categorical_features(df)

print(df_encoded)


BsmtFinType1 - 1234567
BsmtFinType2 - 1234567


In [38]:
estimator = RandomForestRegressor(n_estimators=10, random_state=0)
imputer = IterativeImputer(estimator=estimator, max_iter=10, random_state=0)
imputed_df = df_encoded
imputed_df

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1.0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,...,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,208500.0
1,2.0,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,...,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,181500.0
2,3.0,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,...,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,223500.0
3,4.0,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,...,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,140000.0
4,5.0,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,...,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,...,0.0,40.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,175000.0
1456,1457.0,20.0,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,...,349.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,210000.0
1457,1458.0,70.0,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,275.0,...,0.0,60.0,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,266500.0
1458,1459.0,20.0,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,...,366.0,0.0,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,142125.0


Прекращаем работать с категориальными данными и переходим к численным

In [ ]:
estimator = RandomForestRegressor(n_estimators=10, random_state=0)
imputer = IterativeImputer(estimator=estimator, max_iter=10, random_state=0)
imputed_data = imputer.fit_transform(df_numeric)
imputed_data_test_numeric = imputer.fit_transform(df_numeric)
imputed_df_test_numeric = pd.DataFrame(imputed_data_test_numeric, columns=df_numeric.columns)
imputed_df = pd.DataFrame(imputed_data, columns=df_numeric.columns)

In [57]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

data = imputed_df.copy()

contamination = 0.03  # предполагаемая доля выбросов
model = IsolationForest(contamination=contamination, random_state=42)
model.fit(data)

predictions = model.predict(data)

# Замена выбросов на NaN
df_no_outliers = data.copy()
# Преобразуем 1D предсказания в маску для замены выбросов
mask = predictions == -1
for column in df_no_outliers.columns:
    df_no_outliers.loc[mask, column] = np.nan

df_no_outliers

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1.0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,...,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,208500.0
1,2.0,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,...,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,181500.0
2,3.0,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,...,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,223500.0
3,4.0,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,...,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,140000.0
4,5.0,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,...,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,...,0.0,40.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,175000.0
1456,1457.0,20.0,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,...,349.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,210000.0
1457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,1459.0,20.0,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,...,366.0,0.0,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,142125.0


In [ ]:
df_no_outliers

In [7]:
print("Оригинальный DataFrame:")
print(df)

df_no_outliers = replace_outliers_with_nan(df)

print("\nDataFrame после замены выбросов на NaN:")
df_no_outliers

Оригинальный DataFrame:
        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities 

ValueError: Input X contains NaN.
OneClassSVM does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values